In [8]:
 ! pip install selenium

In [38]:
## 라이브러리 및 웹드라이브 설정

import pandas as pd
from bs4 import BeautifulSoup 
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from matplotlib import font_manager, rc
rc('font', family='AppleGothic')
import FinanceDataReader as web
from datetime import date, timedelta
import matplotlib.pyplot as plt
import datetime
import urllib.parse
import time
import math

%matplotlib inline

## 웹 드라이브 설정

options = webdriver.ChromeOptions()
options.add_argument("window-size=1920*1080")
driver_loc = "../../../selenium/chromedriver" ## 제출전 경로확인
driver = webdriver.Chrome(executable_path=driver_loc,options=options)

targetUrl = "https://finance.naver.com/"
driver.get(targetUrl)

In [39]:
## 입력받은 데이터의 종목 코드를 확인
MARKET_CODE_DICT = {
    'kospi': 'stockMkt',
    'kosdaq': 'kosdaqMkt',
    'konex': 'konexMkt'
}
DOWNLOAD_URL = 'https://kind.krx.co.kr/corpgeneral/corpList.do'
MARKET_CODE_DICT

{'kospi': 'stockMkt', 'kosdaq': 'kosdaqMkt', 'konex': 'konexMkt'}

In [40]:
##종목 코드에 맞춰서 zfill=자릿수만큼 앞에 0으로 채워주는 함수

def zeroFill(columnValue):
    columnValue = str(columnValue)
    outValue = columnValue.zfill(6)
    return outValue

In [41]:
## 상장폐지가 아닌 상장사들만 조회
## input되는 key값에 맞는 value 값 전체가 return 됨

def stock_list(market=None, delisted=False):
    params = {'method': 'download'}

    if market.lower() in MARKET_CODE_DICT:
        params['marketType'] = MARKET_CODE_DICT[market]
        print(market.lower()+" market key is exist")

    else:
        print("invalid market")

    params_string = urllib.parse.urlencode(params)
    request_url = DOWNLOAD_URL+"?"+params_string
    df = pd.read_html(request_url)[0]
    df["종목코드"] = df.종목코드.apply(zeroFill) 
    return df

In [42]:
## 유저가 입력한 주식종목의 코드 가져오기
def get_stock_number(stock_Name):
    stocks = stock_list('kospi')
    stocksRow = stocks.loc[stocks.회사명==stock_Name]
    if(len(stocksRow)<=0):
        print("can not found stock_name")
    else:
        code_number = stocksRow.iloc[0]['종목코드']
        code_number = str(code_number)
        return code_number

In [43]:
def crawling_with_code(stock_code):
    ## 분기별 기업 실적 정보 크롤링
    ##검색창에 마우스 커서클릭
    searchXpath = "/html/body/div[2]/div[1]/div/div[2]/form/fieldset/div/div[1]/input"
    searchClick = driver.find_element_by_xpath(searchXpath)
    searchClick.click() 

    ##종목 키워드 검색
    stockNameSearch = driver.find_element_by_xpath(searchXpath)
    searchKeyword = stock_code
    stockNameSearch.send_keys(searchKeyword)
    stockNameSearch.send_keys(Keys.ENTER)

    ##분기별 기업실적분석까지 스크롤내리기
    time.sleep(3)
    driver.execute_script("window.scrollTo(0, 1200)")
    time.sleep(3)

    targetUrl = driver.current_url 
    resp = requests.get(url=targetUrl)
    html = resp.text
    htmlObj = BeautifulSoup(html, "html.parser",from_encoding='utf-8')
    
    
    tables=htmlObj.select('table')
#     time.sleep(2)
    table = tables[3]
#     time.sleep(2)

    return change_type_crwaling_code(table)

In [44]:
## 시가총액 정보
def crawling_with_marketcap_info(stock_code):
    time.sleep(2)

    ## 분기별 기업 실적 정보 크롤링
    ##검색창에 마우스 커서클릭
    searchXpath = "/html/body/div[2]/div[1]/div/div[2]/form/fieldset/div/div[1]/input"
    searchClick = driver.find_element_by_xpath(searchXpath)
    searchClick.click()
    time.sleep(2)

    ##종목 키워드 검색
    stockNameSearch = driver.find_element_by_xpath(searchXpath)
    searchKeyword = stock_code
    time.sleep(2)
    stockNameSearch.send_keys(searchKeyword)
    stockNameSearch.send_keys(Keys.ENTER)

    ##분기별 기업실적분석까지 스크롤내리기
    driver.execute_script("window.scrollTo(0, 1200)")
    time.sleep(3)
    
    targetUrl = driver.current_url 
    resp = requests.get(url=targetUrl)
    html = resp.text
    htmlObj = BeautifulSoup(html, "html.parser",from_encoding='utf-8')
    
    tables=htmlObj.select('table')
    time.sleep(3)

    table = tables[5]
    time.sleep(3)

    return change_type_crwaling_code(table)

In [45]:
def change_type_crwaling_code(table):
    # 테이블 html 정보를 문자열로 변경하기
    table_html = str(table)

    # 판다스의 read_html 로 테이블 정보 읽기
    table_df_list = pd.read_html(table_html)

    # 데이터프레임 선택하기
    info_table = table_df_list[0]
    return info_table

In [46]:
# 유저가 입력한 주식이름 받아오기
userInputstockName=str(input())

카카오


In [47]:
stock_code=get_stock_number(userInputstockName)
stock_code

kospi market key is exist


'035720'

In [49]:
table_df=crawling_with_code(stock_code)
table_df

C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:203: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


주요재무정보  최근 연간 실적                                 최근 분기 실적            \
       주요재무정보   2018.12   2019.12   2020.12 2021.12(E)   2020.03   2020.06   
       주요재무정보    IFRS연결    IFRS연결    IFRS연결     IFRS연결    IFRS연결    IFRS연결   
0         매출액  24170.00  30701.00  41568.00   56775.00   8684.00   9529.00   
1        영업이익    729.00   2068.00   4559.00    7845.00    882.00    978.00   
2       당기순이익    159.00  -3419.00   1734.00    8630.00    799.00   1452.00   
3       영업이익률      3.02      6.73     10.97      13.82     10.16     10.26   
4        순이익률      0.66    -11.14      4.17      15.20      9.20     15.24   
5   ROE(지배주주)      1.04     -5.81      2.70      11.77     -4.73     -2.67   
6        부채비율     41.45     52.21     60.94        NaN     57.01     49.86   
7        당좌비율    146.51    127.98    162.77        NaN    134.03    144.53   
8         유보율  12219.62  12027.79  13881.01        NaN  12587.68  13665.91   
9      EPS(원)    123.00   -717.00    355.00    1770.00    179.00    320.00   
10     PER(배)    168.01    -42.82    219.25      81.64    -52.50   -152.84   
11     BPS(원)  12979.00  12746.00  14647.00   16327.00  13108.00  14073.00   
12     PBR(배)      1.59      2.41      5.32       8.85      2.37      3.80   
13   주당배당금(원)     25.00     25.00     30.00     106.00       NaN       NaN   
14   시가배당률(%)      0.12      0.08      0.04        NaN       NaN       NaN   
15    배당성향(%)     20.99     -3.46      8.29        NaN       NaN       NaN   

                                             
     2020.09   2020.12   2021.03 2021.06(E)  
      IFRS연결    IFRS연결    IFRS연결     IFRS연결  
0   11004.00  12351.00  12580.00   13461.00  
1    1202.00   1497.00   1575.00    1789.00  
2    1437.00  -1955.00   2399.00    1985.00  
3      10.92     12.12     12.52      13.29  
4      13.06    -15.83     19.07      14.75  
5      -1.33      2.70      5.00        NaN  
6      50.37     60.94     71.56        NaN  
7     148.86    162.77    163.55        NaN  
8   14373.52  13881.01  14508.10        NaN  
9     290.00   -428.00    506.00     445.00  
10   -404.46    219.25    144.96     324.73  
11  14910.00  14647.00  15428.00        NaN  
12      4.89      5.32      6.46        NaN  
13       NaN       NaN       NaN        NaN  
14       NaN       NaN       NaN        NaN  
15       NaN       NaN       NaN        NaN

In [51]:
yearData2018=table_df['최근 연간 실적']['2018.12']['IFRS연결']
yearData2019=table_df['최근 연간 실적']['2019.12']['IFRS연결']
yearData2020=table_df['최근 연간 실적']['2020.12']['IFRS연결']

seasonData202003=table_df['최근 분기 실적']['2020.03']['IFRS연결']
seasonData202006=table_df['최근 분기 실적']['2020.06']['IFRS연결']
seasonData202009=table_df['최근 분기 실적']['2020.09']['IFRS연결']
seasonData202012=table_df['최근 분기 실적']['2020.12']['IFRS연결']
seasonData202103=table_df['최근 분기 실적']['2021.03']['IFRS연결']
seasonData202106=table_df['최근 분기 실적']['2021.06(E)']['IFRS연결']

#Category=table_df['주요재무정보']['주요재무정보']['주요재무정보']

# 연간데이터
yearData=pd.DataFrame(zip(yearData2018,yearData2019,yearData2020),columns=["2018","2019","2020"])
yearData

,2018,2019,2020
0,24170.00,30701.00,41568.00
1,729.00,2068.00,4559.00
2,159.00,-3419.00,1734.00
3,3.02,6.73,10.97
4,0.66,-11.14,4.17
5,1.04,-5.81,2.70
6,41.45,52.21,60.94
7,146.51,127.98,162.77
8,12219.62,12027.79,13881.01
9,123.00,-717.00,355.00


In [52]:
# 분기데이터
seasonData=pd.DataFrame(zip(seasonData202003,seasonData202006,seasonData202009,seasonData202012,seasonData202103,seasonData202106),
             columns=["202003","202006","202009","202012","202103","202106"])
seasonData

,202003,202006,202009,202012,202103,202106
0,8684.00,9529.00,11004.00,12351.00,12580.00,13461.00
1,882.00,978.00,1202.00,1497.00,1575.00,1789.00
2,799.00,1452.00,1437.00,-1955.00,2399.00,1985.00
3,10.16,10.26,10.92,12.12,12.52,13.29
4,9.20,15.24,13.06,-15.83,19.07,14.75
5,-4.73,-2.67,-1.33,2.70,5.00,NaN
6,57.01,49.86,50.37,60.94,71.56,NaN
7,134.03,144.53,148.86,162.77,163.55,NaN
8,12587.68,13665.91,14373.52,13881.01,14508.10,NaN
9,179.00,320.00,290.00,-428.00,506.00,445.00


In [53]:
seasonData.index=(['매출액', '영업이익', '당기순이익','영업이익률','순이익률','ROE(지배주주)','부채비율','당좌비율','유보율','EPS(원)','PER(배)','BPS(원)','PBR(배)','주당배당금(원)','시가배당률(%)','배당성향(%)'])
yearData.index=(['매출액', '영업이익', '당기순이익','영업이익률','순이익률','ROE(지배주주)','부채비율','당좌비율','유보율','EPS(원)','PER(배)','BPS(원)','PBR(배)','주당배당금(원)','시가배당률(%)','배당성향(%)'])
seasonData

,202003,202006,202009,202012,202103,202106
매출액,8684.00,9529.00,11004.00,12351.00,12580.00,13461.00
영업이익,882.00,978.00,1202.00,1497.00,1575.00,1789.00
당기순이익,799.00,1452.00,1437.00,-1955.00,2399.00,1985.00
영업이익률,10.16,10.26,10.92,12.12,12.52,13.29
순이익률,9.20,15.24,13.06,-15.83,19.07,14.75
ROE(지배주주),-4.73,-2.67,-1.33,2.70,5.00,NaN
부채비율,57.01,49.86,50.37,60.94,71.56,NaN
당좌비율,134.03,144.53,148.86,162.77,163.55,NaN
유보율,12587.68,13665.91,14373.52,13881.01,14508.10,NaN
EPS(원),179.00,320.00,290.00,-428.00,506.00,445.00


In [57]:
# 데이터 전처리
df=seasonData.transpose()
# df['유보율'].plot(figsize=(15,4))
# plt.show()
df

,매출액,영업이익,당기순이익,영업이익률,순이익률,ROE(지배주주),부채비율,당좌비율,유보율,EPS(원),PER(배),BPS(원),PBR(배),주당배당금(원),시가배당률(%),배당성향(%)
202003,8684.0,882.0,799.0,10.16,9.20,-4.73,57.01,134.03,12587.68,179.0,-52.50,13108.0,2.37,NaN,NaN,NaN
202006,9529.0,978.0,1452.0,10.26,15.24,-2.67,49.86,144.53,13665.91,320.0,-152.84,14073.0,3.80,NaN,NaN,NaN
202009,11004.0,1202.0,1437.0,10.92,13.06,-1.33,50.37,148.86,14373.52,290.0,-404.46,14910.0,4.89,NaN,NaN,NaN
202012,12351.0,1497.0,-1955.0,12.12,-15.83,2.70,60.94,162.77,13881.01,-428.0,219.25,14647.0,5.32,NaN,NaN,NaN
202103,12580.0,1575.0,2399.0,12.52,19.07,5.00,71.56,163.55,14508.10,506.0,144.96,15428.0,6.46,NaN,NaN,NaN
202106,13461.0,1789.0,1985.0,13.29,14.75,NaN,NaN,NaN,NaN,445.0,324.73,NaN,NaN,NaN,NaN,NaN


In [56]:
df_year=yearData.transpose()
df=df.fillna(0)
df_year=df_year.fillna(0)
df_year

,매출액,영업이익,당기순이익,영업이익률,순이익률,ROE(지배주주),부채비율,당좌비율,유보율,EPS(원),PER(배),BPS(원),PBR(배),주당배당금(원),시가배당률(%),배당성향(%)
2018,24170.0,729.0,159.0,3.02,0.66,1.04,41.45,146.51,12219.62,123.0,168.01,12979.0,1.59,25.0,0.12,20.99
2019,30701.0,2068.0,-3419.0,6.73,-11.14,-5.81,52.21,127.98,12027.79,-717.0,-42.82,12746.0,2.41,25.0,0.08,-3.46
2020,41568.0,4559.0,1734.0,10.97,4.17,2.70,60.94,162.77,13881.01,355.0,219.25,14647.0,5.32,30.0,0.04,8.29


In [58]:
## 모든 kospi의 등록된 회사의 기본 점수를 100점으로 두고 시작한다.
## 각 기준에 맞추어 해당 점수를 하락시킨다

risk_score = 0
stable_score = 0
month_total = 5
month_for_x = 3 ## 해당 그래프의 데이터가 3달 간격으로 작성

In [59]:
## 부채비율 그래프에서, 3개월 간격으로 부채율 증가 추세를 보이면
for i in range (0,month_total):
    if(((df['부채비율'][i+1]-df['부채비율'][i])/month_for_x)>=0):
        risk_score = risk_score+2
    else:
        continue

## 평균 부채비율이 100보다 클 경우, 위험도 경고
def dept_avg(df):
    if(df['부채비율'].mean()>=100):
        risk_score = risk_score+1
        print("위험 : 평균 부채비율이 높습니다")
        return risk_score
    else:
        return 0

def find_reverse_rate(df):
    if(df['유보율'][len(df)-1]==0):
        rate = df['유보율'][len(df)-2]
    else:
        rate=df['유보율'][len(df)-1] 

    if(rate>=500):
        stable_score += 2
        if(dept_avg(df)>=1):
            risk_score += 1
            print ("위험 : 업계 특성 고려, 유보율 높지만 부채율도 높습니다")
        else:
            print("안정성 높습니다")
    else:
        if(dept_avg(df)>=1):
            risk_score += 2
            print("위험 : 구매시 손해")
        risk_score += 1
        print ("성장 기업일 가능성 상승")

In [60]:
def yearly_growth_rate(df2):    
    growth_rate2019=(int(df2['매출액'][1])-int(df2['매출액'][0]))/int(df2['매출액'][0])*100
    growth_rate2020=(int(df2['매출액'][2])-int(df2['매출액'][1]))/int(df2['매출액'][1])*100
    
    operating_profit2019=int(df2['영업이익'][1])-int(df2['영업이익'][0])
    operating_profit2020=int(df2['영업이익'][2])-int(df2['영업이익'][1])

    net_income2019=int(df2['당기순이익'][1])-int(df2['당기순이익'][0])
    net_income2020=int(df2['당기순이익'][2])-int(df2['당기순이익'][1])
    
    if(growth_rate2019<growth_rate2020):
        print("전년대비 매출 up",round(growth_rate2020,3),"%")
        return1 = "up"
    elif(growth_rate2019>=growth_rate2020):
        print ("매출 down")
        return1 = "down"
        
    if(operating_profit2020>operating_profit2019):
        print("전년대비 영업이익 up")
        if(operating_profit2020<=0):
            print("영업이익(-) : ",operating_profit2020)
            return2 = "down"

        else:
            print("영업이익(+) : ",operating_profit2020)
            return2 = "up"
    else:
        print("전년대비 영업이익 down")
        return2 = "down"
        
    if(int(df2['당기순이익'][2])>int(df2['당기순이익'][1])):
        print("전년대비 당기순이익 up")
        
        if(net_income2020>=200):
            print("당기 순이익 급성장")
            return3 = "up"

        elif(net_income2020>100):
            print("당기 순이익 성장요소 보임")
            return3 = "up"

        else:
            print("당기 순이익 미세하게 성장")
            return3 = "up"

    else:
        print("전년대비 당기순이익 down")
        return3 = "down"

    return return1, return2, return3

In [61]:
## EPS(주당순이익) : 당기 순이익 / 총주식수
def get_EPS(userInputstockName):
    count_stocks=int(crawling_with_marketcap_info(userInputstockName)[1][2])
    netIncome=int(df_year['당기순이익'][2])*100000000
    EPS=netIncome/count_stocks
    return EPS

In [62]:
## 결과 가져오기
# 1) 매출, 영업이익, 당기순이익
result_before=yearly_growth_rate(df_year)

전년대비 매출 up 35.396 %
전년대비 영업이익 up
영업이익(+) :  2491
전년대비 당기순이익 up
당기 순이익 급성장


In [63]:
# 2) 주당 순이익(eps)
stock_num = get_stock_number(userInputstockName)
eps = int(get_EPS(stock_num))

kospi market key is exist


C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:203: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


In [64]:
# 3) 최종 결과

cnt=0
for i in range(0,len(result_before)):
    if(result_before[i]=="down"):
        cnt = cnt+1
    else:
        continue
        
def find_result(result_before):
    if cnt>=2 & eps <= 0:
        result="구매 비추천이나, 종목별(중공업, 반도체)특성의 영향이 클 가능성 높음"
    elif(cnt>=2 & eps > 0):
        result = "구매 비추천"
    elif(cnt<2 & eps >= 50):
        result = "구매 추천, 기대주"
    else:
        result = "구매추천"

    return result
find_result(cnt)

'구매추천'